# Setup workflow environment

In [1]:
using Pkg; Pkg.activate(@__DIR__)

import TulipaIO as TIO
import TulipaEnergyModel as TEM
import TulipaClustering as TC
using DuckDB
using DataFrames
using Plots

# For Win. system to fix the KaTex parse error in Jupyter Notebook
Base.show(stdout, ::MIME"text/latex", df::DataFrame) = show(stdout, MIME("text/plain"), df)

#= For utility functions
    - print_annual_total_prod(DBconnection, years...)
=#
include("./util.jl");

  Activating project at `~/Projects/VintageDemo`


# Multi-year investment model instances

- Milestone years: 2030, 2040 and 2050.
- The system has 30GW initial wind capacity built in 2025, the model can choose to invest in wind in three milestone years: 2030, 2040, 2050.
- unused tables: `flow-both.csv`, `flows-profiles.csv`

## 1: Without unit vintage

> Note: the `output_dir` must exist/(be created) beforehand 

### 1.1 Build and run the model instance

In [2]:
connection = DBInterface.connect(DuckDB.DB)
input_dir = "model-instance-Tulipa/inputs-no-vintage"
output_dir = joinpath(@__DIR__, "model-instance-Tulipa/outputs-no-vintage")

# Always build a new result directory
rm(output_dir, force=true, recursive=true) 
mkdir(output_dir);

# Create the connection and prepare input data
connection_no_vintage = DBInterface.connect(DuckDB.DB)
TIO.read_csv_folder(connection_no_vintage, input_dir)
TEM.populate_with_defaults!(connection_no_vintage)

# Run the model instance
multiyear_no_vintage = TEM.run_scenario(
    connection_no_vintage;
    model_parameters_file = joinpath(@__DIR__, input_dir, "model-parameters.toml"),
    output_folder = output_dir, 
    # model_file_name = joinpath(output_dir, "model.lp"),
    show_log=false,
    log_file = joinpath(output_dir, "model-log_no-vintage.txt")
)

EnergyProblem:
  - Model created!
    - Number of variables: 315363
    - Number of constraints for variable bounds: 315363
    - Number of structural constraints: 341640
  - Model solved!
    - Termination status: OPTIMAL
    - Objective value: 1.2493185954028405e6


### 1.2 Key results

#### Capacity

Check the "initial capacity" 

- in this case, we will not be able to differentiate units built in other years (than milestone years), 
- they will simply be considered the same as the units built in the milestone year.

In [3]:
# initial wind capacity
filter(row -> row.asset=="wind", TIO.get_table(connection_no_vintage, "asset_both"))

Row,asset,milestone_year,commission_year,decommissionable,initial_units,initial_storage_units
,String,Int32,Int32,Bool,Float64,Float64
1,wind,2030,2030,false,30.0,0.0
2,wind,2040,2040,false,30.0,0.0
3,wind,2050,2050,false,30.0,0.0


In [4]:
# invested capacity
filter(row -> row.asset=="wind", TIO.get_table(connection_no_vintage, "var_assets_investment"))

Row,id,asset,milestone_year,investment_integer,capacity,investment_limit,solution
,Int64,String,Int32,Bool,Float64,Float64,Float64
1,1,wind,2030,true,1.0,107.567,107.0
2,2,wind,2040,true,1.0,186.156,166.0
3,3,wind,2050,true,1.0,217.081,34.0


#### Annual productions & total system cost

In [5]:
print_annual_total_prod(connection_no_vintage, 2030, 2040, 2050)
println("Total system cost: $(round(multiyear_no_vintage.objective_value/1000, digits=2)) Billion €")

2030s
	 wind prodution: 665.62 TWh p.a.

	 market supply: 313.29 TWh p.a.
2040s
	 wind prodution: 1413.94 TWh p.a.

	 market supply: 516.3 TWh p.a.
2050s
	 wind prodution: 1489.76 TWh p.a.

	 market supply: 753.49 TWh p.a.
Total system cost: 1249.32 Billion €


## 2: Explicit unit vintage

- Difference from case 1: `asset.csv`, `asset-milestone.csv`, `asset-commission.csv`, `asset-both`, `assets-profiles.csv`, `flow*.csv`
- In any milestone year, only the tech. vintage of the same year is available for investment (explicitly regulated by setting the `investable` parameter in `asset-milestone.csv`)

### 2.1 Build and run the model instance

In [2]:
# Define and build the input output directories
input_dir = "model-instance-Tulipa/inputs-vintage-standard"
output_dir = joinpath(@__DIR__, "model-instance-Tulipa/outputs-vintage-standard")

# Always build a new result directory
rm(output_dir, force=true, recursive=true) 
mkdir(output_dir);

# Create the connection and prepare input data
connection_vintage_standard = DBInterface.connect(DuckDB.DB)
TIO.read_csv_folder(connection_vintage_standard, input_dir)
TEM.populate_with_defaults!(connection_vintage_standard)

# Run the model instance
multiyear_vintage_standard = TEM.run_scenario(
    connection_vintage_standard;
    model_parameters_file = joinpath(@__DIR__, input_dir, "model-parameters.toml"),
    output_folder = output_dir, 
    # model_file_name = joinpath(output_dir, "model.lp"),
    show_log=false,
    log_file = joinpath(output_dir, "model-log_vintage-standard.txt")
)

EnergyProblem:
  - Model created!
    - Number of variables: 788403
    - Number of constraints for variable bounds: 788403
    - Number of structural constraints: 420480
  - Model solved!
    - Termination status: OPTIMAL
    - Objective value: 1.2205387507129149e6


### 2.2 Key results

#### Capacity

In [3]:
# initial wind capacity
filter(row -> occursin("wind", row.asset) && row.initial_units != 0.0, TIO.get_table(connection_vintage_standard, "asset_both"))

Row,asset,milestone_year,commission_year,decommissionable,initial_units,initial_storage_units
,String,Int32,Int32,Bool,Float64,Float64
1,wind25,2030,2030,false,30.0,0.0
2,wind25,2040,2040,false,30.0,0.0
3,wind25,2050,2050,false,30.0,0.0


In [4]:
# invested capacity
filter(row -> occursin("wind", row.asset) && row.solution != 0.0, TIO.get_table(connection_vintage_standard, "var_assets_investment"))

Row,id,asset,milestone_year,investment_integer,capacity,investment_limit,solution
,Int64,String,Int32,Bool,Float64,Float64,Float64
1,1,wind30,2030,true,1.0,107.567,107.0
2,2,wind40,2040,true,1.0,186.156,186.0
3,3,wind50,2050,true,1.0,217.081,105.0


#### Annual productions & total system cost

In [5]:
print_annual_total_prod(connection_vintage_standard, 2030, 2040, 2050)
println("Total system cost: $(round(multiyear_vintage_standard.objective_value/1000, digits=2)) Billion €")

2030s
	 wind prodution: 661.08 TWh p.a.

	 market supply: 317.83 TWh p.a.
2040s
	 wind prodution: 1507.38 TWh p.a.

	 market supply: 422.86 TWh p.a.
2050s
	 wind prodution: 1815.86 TWh p.a.

	 market supply: 427.39 TWh p.a.
Total system cost: 1220.54 Billion €


## 3: Compact unit vintage

- Difference from case 1: `asset.csv` (only `investment_method`), `asset-commission.csv`, `asset-both`
- In any milestone year, only the tech. vintage of the same year is available for investment (assumed with setting `investment_method=compact`)
- Resulted investment in 2050 differ significantly from the no vintage case because the wind of 2020 vintage (wind commissioned in 2020, defined in `asset-commission.csv`) uses the default availability of `1.0` over the milestone year 2050 due to the lack of availability value is given for 2050 for this vintage (`availability-wind2020` records in `profiles-rep-periods.csv`, which is assigned to the wind commissioned in 2020 in `assets-profiles.csv`)

### 3.1 Build and run the model instance

In [2]:
# Define and build the input output directories
input_dir = "model-instance-Tulipa/inputs-vintage-compact"
output_dir = joinpath(@__DIR__, "model-instance-Tulipa/outputs-vintage-compact")

# Always build a new result directory
rm(output_dir, force=true, recursive=true) 
mkdir(output_dir);

# Create the connection and prepare input data
connection_vintage_compact = DBInterface.connect(DuckDB.DB)
TIO.read_csv_folder(connection_vintage_compact, input_dir)
TEM.populate_with_defaults!(connection_vintage_compact)

# Run the model instance
multiyear_vintage_compact = TEM.run_scenario(
    connection_vintage_compact;
    model_parameters_file = joinpath(@__DIR__, input_dir, "model-parameters.toml"),
    output_folder = output_dir, 
    # model_file_name = joinpath(output_dir, "model.lp"),
    show_log=false,
    log_file = joinpath(output_dir, "model-log_vintage-compact.txt")
)

EnergyProblem:
  - Model created!
    - Number of variables: 315363
    - Number of constraints for variable bounds: 315363
    - Number of structural constraints: 341640
  - Model solved!
    - Termination status: OPTIMAL
    - Objective value: 1.2205387507129149e6


### 3.2 Key results

#### Capacity

- Units built in different years are explicitly listed, 
- meaning that their corresponding profiles are also considered.

In [3]:
# initial wind capacity
filter(row -> row.asset=="wind" && row.initial_units != 0, TIO.get_table(connection_vintage_compact, "asset_both"))

Row,asset,milestone_year,commission_year,decommissionable,initial_units,initial_storage_units
,String,Int32,Int32,Bool,Float64,Float64
1,wind,2030,2025,false,30.0,0.0
2,wind,2040,2025,false,30.0,0.0
3,wind,2050,2025,false,30.0,0.0


In [4]:
# invested capacity of the wind commissioned in the milestone year
filter(row -> row.asset=="wind", TIO.get_table(connection_vintage_compact, "var_assets_investment"))

Row,id,asset,milestone_year,investment_integer,capacity,investment_limit,solution
,Int64,String,Int32,Bool,Float64,Float64,Float64
1,1,wind,2030,true,1.0,107.567,107.0
2,2,wind,2040,true,1.0,186.156,186.0
3,3,wind,2050,true,1.0,217.081,105.0


#### Annual productions & total system cost

In [5]:
print_annual_total_prod(connection_vintage_compact, 2030, 2040, 2050)
println("Total system cost: $(round(multiyear_vintage_compact.objective_value/1000, digits=2)) Billion €")

2030s
	 wind prodution: 661.08 TWh p.a.

	 market supply: 317.83 TWh p.a.
2040s
	 wind prodution: 1507.38 TWh p.a.

	 market supply: 422.86 TWh p.a.
2050s
	 wind prodution: 1815.86 TWh p.a.

	 market supply: 427.39 TWh p.a.
Total system cost: 1220.54 Billion €


# Ancillary

In [ ]:
# # show all input/output tables
# println(TIO.show_tables(connection_no_vintage).name)
# TIO.show_tables(connection_no_vintage)

# Testing ...

In [ ]:
# using CSV
# show all input/output tables
# println(TIO.show_tables(connection_vintage_standard).name)
# CSV.write(joinpath(output_dir,"model_info.csv"), TIO.show_tables(connection_vintage_compact))

In [ ]:
# # show specific profiles
# filter(
#     row -> occursin("wind", row.profile_name) && row.year == 2050,
#     TIO.get_table(connection_vintage_standard, "profiles_rep_periods")
# )

In [ ]:
function plot_input_profiles(conn::DuckDB.DB, ::Val{:wind}, year::Int)
    plot()
    profiles = filter(
        row -> occursin(String(:wind), row.profile_name) && row.year == year,
        TIO.get_table(conn, "profiles_rep_periods")
    )

    for pname in unique(profiles.profile_name)
        subdf = profiles[profiles.profile_name .== pname, :]
        plot!(subdf.value, label="$(pname), year $year")
    end
    xlabel!("Time")
    ylabel!("Capacity factor")
end

plot_input_profiles(connection_vintage_standard, Val(:wind), 2030)

In [ ]:
# @show fieldnames(TEM.EnergyProblem)
# @show dump(TEM.EnergyProblem)
# var_flow = multiyear_vintage_compact.variables[:flow]
# fieldnames(typeof(var_flow))
# typeof(var_flow.indices)
# var_flow.indices.tbl[:id]
# multiyear_no_vintage.expressions[:available_asset_units_simple_method]
